In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ultralytics -q
!pip install tqdm -q

In [3]:
import sys
sys.path.append('/content/drive/Othercomputers/Doug Desktop/ms-aai-521-final-project/src')

In [6]:
import logging
from pathlib import Path

from tqdm import tqdm

from Preprocessor import Preprocessor
from TrainingManager import TrainingManager
from YOLODetector import YOLODetector
from config import OUTPUT_DIR, YOLO_MODEL, YOLO_CONFIDENCE_THRESHOLD, YOLO_BATCH_SIZE
from visualize import create_detection_video

In [7]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load dataset
logger.info("Loading dataset")
preprocessor = Preprocessor()
clips = preprocessor.prepare_dataset()

# Train the model
logger.info("Starting training process")
trainer = TrainingManager()

# Split dataset
train_clips, val_clips, test_clips = trainer.prepare_training(clips)

# Train model
trainer.train(train_clips, val_clips)
logger.info("Training complete!")

# Use the fine-tuned model for evaluation
model_path = Path(OUTPUT_DIR) / "training_data/train/finetune/weights/best.pt"
if not model_path.exists():
    logger.warning("No fine-tuned model found, using pre-trained model")
    model_path = YOLO_MODEL

detector = YOLODetector(
    model_name=str(model_path),
    conf_threshold=YOLO_CONFIDENCE_THRESHOLD
)

for clip in tqdm(clips):
    logger.info(f"Processing clip {clip.video_id}")

    detections = detector.detect_video(
        clip.frames,
        batch_size=YOLO_BATCH_SIZE
    )

    output_path = Path(OUTPUT_DIR) / 'detections' / f'{clip.video_id}_detections.mp4'
    logger.info(f"Creating visualization video: {output_path}")

    create_detection_video(
        frames=clip.frames,
        detections=detections,
        output_path=str(output_path)
    )

logger.info("Done!")

Ultralytics 8.3.36 🚀 Python-3.10.12 torch-2.5.1+cpu CPU (Intel Xeon 2.00GHz)


RepresenterError: ('cannot represent an object', <torch.utils.data.dataloader.DataLoader object at 0x7ad1e282b160>)